<a href="https://colab.research.google.com/github/sam757575/collab/blob/main/Vertex_AI_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ML Experiment Tracking with Vertex AI

Great to have you here

* 📖 Article: https://medium.com/google-cloud/machine-learning-experiment-tracking-with-vertex-ai-8406f8d44376
* 🎥 Video: https://www.youtube.com/watch?v=a_YXZ5UltkU

Your feedback and questions are highly appreciated. <br>You can find me on Twitter [@HeyerSascha](https://twitter.com/HeyerSascha) or connect with me via [LinkedIn](https://www.linkedin.com/in/saschaheyer/). <br>Even better, subscribe to my [YouTube](https://www.youtube.com/channel/UC--Sm3D-rqCUeLXmraypdPQ) channel ❤️.

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/a_YXZ5UltkU" frameborder="0" allowfullscreen></iframe>')

## Authentication

In [ ]:
from google.colab import auth
auth.authenticate_user()

## Dependencies

In [ ]:
! pip install google-cloud-aiplatform==1.18.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import google.cloud.aiplatform as aiplatform
# if you get an error just restart the runtime and run again

import time

## Initialize Experiment

In [ ]:
aiplatform.init(
       project='sascha-playground-doit',
       location='us-central1',
       experiment='experiment-demo',
    )

## Add TensorBoard Instance to the experiment (required only once)

In [ ]:
experiment_tensorboard = aiplatform.Tensorboard.create()
experiment_tensorboard_name = experiment_tensorboard.gca_resource.name
print("TensorBoard resource name:", experiment_tensorboard_name)

# sample
# experiment_tensorboard='projects/sascha-playground-doit/locations/us-central1/tensorboards/1834521956804722688'

Creating Tensorboard


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Creating Tensorboard


Create Tensorboard backing LRO: projects/234439745674/locations/us-central1/tensorboards/4164571814015533056/operations/912107911000358912


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:Create Tensorboard backing LRO: projects/234439745674/locations/us-central1/tensorboards/4164571814015533056/operations/912107911000358912


Tensorboard created. Resource name: projects/234439745674/locations/us-central1/tensorboards/4164571814015533056


...

To use this Tensorboard in another session:


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:To use this Tensorboard in another session:


tb = aiplatform.Tensorboard('projects/234439745674/locations/us-central1/tensorboards/4164571814015533056')


INFO:google.cloud.aiplatform.tensorboard.tensorboard_resource:tb = aiplatform.Tensorboard('projects/234439745674/locations/us-central1/tensorboards/4164571814015533056')


TensorBoard resource name: projects/234439745674/locations/us-central1/tensorboards/4164571814015533056


In [ ]:
aiplatform.init(
       project='sascha-playground-doit',
       location='us-central1',
       experiment='experiment-demo',
       experiment_tensorboard=experiment_tensorboard
    )

## Start Run

In [ ]:
aiplatform.start_run("run-{}".format(int(time.time())))

Associating projects/234439745674/locations/us-central1/metadataStores/default/contexts/experiment-demo-run-1668692828 to Experiment: experiment-demo


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/234439745674/locations/us-central1/metadataStores/default/contexts/experiment-demo-run-1668692828 to Experiment: experiment-demo


## Log parameter

In [ ]:
metaparams = {}
metaparams["units"] = 64
metaparams["batch size"] = 32
metaparams["note"] = "team 5"
aiplatform.log_params(metaparams)

## Log metrics

### Summary Metrics

In [ ]:
metrics = {}
metrics["f1"] = 0.84
aiplatform.log_metrics(metrics)

### Time Series Metrics

In [ ]:
aiplatform.log_time_series_metrics({"mse": 2500.00, "rmse": 50.00})
aiplatform.log_time_series_metrics({"mse": 2600.00, "rmse": 60.00})
aiplatform.log_time_series_metrics({"mse": 2700.00, "rmse": 70.00})

### Classification Metrics

In [ ]:
from sklearn.metrics import confusion_matrix

labels = ["ant", "bird", "cat"]

y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]


classification_metrics = {
        "matrix": confusion_matrix(y_true, y_pred, labels=labels).tolist(),
        "labels": labels,
    }

print(classification_metrics)

aiplatform.log_classification_metrics(
        labels=classification_metrics["labels"],
        matrix=classification_metrics["matrix"],
        display_name="my-confusion-matrix",
    )

{'matrix': [[2, 0, 0], [0, 0, 1], [1, 0, 2]], 'labels': ['ant', 'bird', 'cat']}


## Artifacts

In [ ]:
training_data_artifact = aiplatform.Artifact.create(
        schema_title='system.Dataset',
        uri='gs://doit-vertex-demo/higgs/training.csv',
        display_name='data')

with aiplatform.start_execution(
       schema_title="system.ContainerExecution",
       display_name='training'
    ) as execution:

        execution.assign_input_artifacts([training_data_artifact])

        # do the training and store the model as output artifact

        model_artifact = aiplatform.Artifact.create(
        schema_title='system.Model',
        uri='gs://doit-vertex-demo/higgs/model',
        display_name='model')

        execution.assign_output_artifacts([model_artifact])

## End experiment run

In [ ]:
aiplatform.end_run()

## Compare expeirment and runs (you can also compare across experiments)

In [ ]:
experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == 'experiment-demo']
experiment_df

,experiment_name,run_name,run_type,state,param.units,param.note,param.batch size,metric.f1,time_series_metric.mse,time_series_metric.rmse
0,experiment-demo,run-1668692828,system.ExperimentRun,RUNNING,64.0,team 5,32.0,0.84,2700.0,70.0


## Get artifcats for experiment run

In [ ]:
experiment_run = aiplatform.ExperimentRun(
        run_name='run-1668692828',
        experiment='experiment-demo',
)

print(experiment_run.get_artifacts())
print(experiment_run.get_metrics())
print(experiment_run.get_params())
print(experiment_run.get_time_series_data_frame())
print(experiment_run.get_classification_metrics())

resource name: projects/234439745674/locations/us-central1/metadataStores/default/artifacts/ffdc36ec-19e1-432e-9114-b335e3f94bd7
uri: gs://doit-vertex-demo/higgs/model
schema_title:system.Model, <google.cloud.aiplatform.metadata.artifact.Artifact object at 0x7ff19d77b4d0> 
resource name: projects/234439745674/locations/us-central1/metadataStores/default/artifacts/49e0a4a7-b5af-41e7-ab7a-767928ee6059
uri: gs://doit-vertex-demo/higgs/training.csv
schema_title:system.Dataset, <google.cloud.aiplatform.metadata.artifact.Artifact object at 0x7ff19d7758d0> 
resource name: projects/234439745674/locations/us-central1/metadataStores/default/artifacts/a1113415-c4ba-4dec-93b1-82dfbf0bfad9
uri: 
schema_title:google.ClassificationMetrics, <google.cloud.aiplatform.metadata.artifact.Artifact object at 0x7ff19d836710> 
resource name: projects/234439745674/locations/us-central1/metadataStores/default/artifacts/experiment-demo-run-1668692828-tb-run
uri: https://us-central1-aiplatform.googleapis.com/v1/pr